# CAISO Energy Storage Bid Data Scraping


<b>Author:</b> Neal Ma<br/>
<b>Creation Date:</b> September 11, 2024

<b>Description:</b> CAISO has released daily energy storage reports here (https://www.caiso.com/library/daily-energy-storage-reports) since August 4, 2022. These reports include extremely useful data but the data itself is not readily accessible. This notebook extracts that data and saves it to a local parquet file to read into a pandas dataframe for easy query and access.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# define all URLs and dates to search
BASE_URL = "https://www.caiso.com/documents/dailyenergystoragereport"
DATE_FORMAT = "%b%d-%Y"
EXTENSION = ".html"

ALT_BASE_URL = "https://www.caiso.com/documents/daily-energy-storage-report-"
ALT_DATE_FORMAT = "%b-%d-%Y"

ALT_2_DATE_FORMAT = "%b-%d%Y"

START_DATE = "2022-07-31"  # NOTE: This is the earliest date with available data
END_DATE = "2024-10-01"  # NOTE: This can be changed but just needs to be verified

HTML_STORAGE_PATH = "./data/CAISO_ES_HTML"
BID_STORAGE_PATH = "./data/ES_BIDS"
DAY_MINUTES = 24 * 60

In [3]:
# build the directory to store html pages
if not os.path.exists(HTML_STORAGE_PATH):
    os.makedirs(HTML_STORAGE_PATH)

# build the directory to store the bids table
if not os.path.exists(BID_STORAGE_PATH):
    os.makedirs(BID_STORAGE_PATH)

In [4]:
date_range = pd.date_range(start=START_DATE, end=END_DATE)
date_strings = date_range.strftime(DATE_FORMAT)
alt_date_strings = date_range.strftime(ALT_DATE_FORMAT)
alt_2_date_strings = date_range.strftime(ALT_2_DATE_FORMAT)

In [5]:
# iterate through all dates and datestrings to find valid dates
failed_requests = []
for date_str, alt_date_str, alt_2_date_str in zip(
    date_strings, alt_date_strings, alt_2_date_strings
):
    html_str = None
    if (page := requests.get(BASE_URL + date_str + EXTENSION)).status_code == 200:
        html_str = BeautifulSoup(page.content, "html.parser")
    elif (
        page := requests.get(ALT_BASE_URL + alt_date_str + EXTENSION)
    ).status_code == 200:
        html_str = BeautifulSoup(page.content, "html.parser")
    elif (
        page := requests.get(ALT_BASE_URL + alt_2_date_str + EXTENSION)
    ).status_code == 200:
        html_str = BeautifulSoup(page.content, "html.parser")
    else:
        failed_requests.append(date_str)
    # save html_str to file with the date_str
    html_str = str(html_str)

    # keep lines in html_str between and including the line starting with 'var tot_energy_ifm = [' and with the final line starting with '	bid_rtpd_neg_hybrid_11 = ['
    lines = html_str.split("\n")
    start_index = next(
        i
        for i, line in enumerate(lines)
        if line.strip().startswith("var tot_energy_ifm = [")
    )
    end_index = next(
        i
        for i, line in enumerate(lines)
        if line.strip().startswith("bid_rtpd_neg_hybrid_11 = [")
    )
    html_str = "".join(lines[start_index : end_index + 1])

    f = open(HTML_STORAGE_PATH + "/" + date_str + ".html", "w")
    f.write(str(html_str))
    f.close()

print(failed_requests)

[]


In [5]:
# define data names and time granularity in minutes
key_dict = {
    "tot_energy_ifm": 5,
    "tot_energy_ruc": 5,
    "tot_energy_rtpd": 5,
    "tot_energy_rtd": 5,
    "tot_charge_ifm": 5,
    "tot_charge_ruc": 5,
    "tot_charge_rtpd": 5,
    "tot_charge_rtd": 5,
    "as_ru_ifm": 60,
    "as_rd_ifm": 60,
    "as_sr_ifm": 60,
    "as_nr_ifm": 60,
    "as_ru_rtpd": 15,
    "as_rd_rtpd": 15,
    "as_sr_rtpd": 15,
    "as_nr_rtpd": 15,
    "tot_energy_hybrid_ifm": 5,
    "tot_energy_hybrid_ruc": 5,
    "tot_energy_hybrid_rtpd": 5,
    "tot_energy_hybrid_rtd": 5,
    "tot_charge_hybrid_ifm": 5,
    "tot_charge_hybrid_ruc": 5,
    "tot_charge_hybrid_rtpd": 5,
    "tot_charge_hybrid_rtd": 5,
    "as_ru_hybrid_ifm": 60,
    "as_rd_hybrid_ifm": 60,
    "as_sr_hybrid_ifm": 60,
    "as_nr_hybrid_ifm": 60,
    "as_ru_hybrid_rtpd": 15,
    "as_rd_hybrid_rtpd": 15,
    "as_sr_hybrid_rtpd": 15,
    "as_nr_hybrid_rtpd": 15,
}

# some larger labels to add
prefixes = ["bid_ifm", "bid_rtpd"]
postfixes = ["_pos_", "_neg_"]

for prefix in prefixes:
    segments = 60 if prefix == "bid_ifm" else 15
    for postfix in postfixes:
        string_prefix = prefix + postfix
        seg_ss_str = string_prefix + "ss"
        key_dict[seg_ss_str] = segments
        for i in range(11):
            seg_str = string_prefix + str(i + 1)
            key_dict[seg_str] = segments

In [17]:
def extract_ES_data(
    html_file_name: str = None, key_dict: dict = None, start_time: pd.Timestamp = None
):
    """Extracts the relevant data from the key_dict placing results into a pandas dataframe as:
    datetime | key | value
    ----------------------
             |     |

    :param html_file_name: a string containing the file name, including file path, to the file of interest
    :param key_dict: a dictionary of keys and expected timescale granularity (in minutes)
    :param start_time: a pandas Timestamp object indicating the time at the start of the html file
    :return: a pandas dataframe as described above
    """
    f = open(html_file_name, "r")
    # lines = f.readlines()
    file_string = f.read().strip()
    lines = [line + "]" for line in file_string.split("]")]
    data = []

    # iterate through keys and find line that contains the key
    for key in key_dict:
        key_found = False
        timeseries = pd.date_range(
            start=start_time,
            end=start_time + pd.Timedelta(days=1),
            freq=str(key_dict[key]) + "min",
        )
        for line in lines:
            if key in line:
                key_found = True
                line_split = line.split("[")[1].split("]")[0].split(",")
                line_values = [
                    float(val) if "NA" not in val else np.nan for val in line_split
                ]
                for ts, val in zip(timeseries[:-1], line_values):
                    data.append([ts, key, val])
                break
        if not key_found:
            for ts in timeseries[:-1]:
                data.append([ts, key, np.nan])

    df = pd.DataFrame(columns=["datetime", "key", "value"], data=data)

    return df


df = extract_ES_data(
    "./data/CAISO_ES_HTML/Mar12-2023.html", key_dict, pd.Timestamp("Mar12-2023")
)

print(df)

                datetime              key  value
0    2023-03-12 00:00:00   tot_energy_ifm  299.0
1    2023-03-12 00:05:00   tot_energy_ifm  299.0
2    2023-03-12 00:10:00   tot_energy_ifm  299.0
3    2023-03-12 00:15:00   tot_energy_ifm  299.0
4    2023-03-12 00:20:00   tot_energy_ifm  299.0
...                  ...              ...    ...
8443 2023-03-12 22:45:00  bid_rtpd_neg_11    0.0
8444 2023-03-12 23:00:00  bid_rtpd_neg_11    0.0
8445 2023-03-12 23:15:00  bid_rtpd_neg_11    0.0
8446 2023-03-12 23:30:00  bid_rtpd_neg_11    0.0
8447 2023-03-12 23:45:00  bid_rtpd_neg_11    0.0

[8448 rows x 3 columns]


In [18]:
test_df = extract_ES_data(
    HTML_STORAGE_PATH + "/" + "Sep26-2024.html", key_dict, pd.Timestamp("Sep26-2024")
)
test_df = test_df.pivot(index="datetime", columns="key", values="value")
test_df

key,as_nr_hybrid_ifm,as_nr_hybrid_rtpd,as_nr_ifm,as_nr_rtpd,as_rd_hybrid_ifm,as_rd_hybrid_rtpd,as_rd_ifm,as_rd_rtpd,as_ru_hybrid_ifm,as_ru_hybrid_rtpd,...,tot_charge_rtpd,tot_charge_ruc,tot_energy_hybrid_ifm,tot_energy_hybrid_rtd,tot_energy_hybrid_rtpd,tot_energy_hybrid_ruc,tot_energy_ifm,tot_energy_rtd,tot_energy_rtpd,tot_energy_ruc
datetime,,,,,,,,,,,,,,,,,,,,,
2024-09-26 00:00:00,0.0,0.0,52.0,41.0,0.0,0.0,-510.0,-510.0,0.0,0.0,...,7349.0,5296.0,0.0,-1.0,1.0,0.0,-3.0,-168.0,-50.0,-3.0
2024-09-26 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7349.0,5296.0,0.0,-1.0,1.0,0.0,-3.0,-295.0,-50.0,-3.0
2024-09-26 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7349.0,5296.0,0.0,-1.0,1.0,0.0,-3.0,-42.0,-50.0,-3.0
2024-09-26 00:15:00,NaN,0.0,NaN,41.0,NaN,0.0,NaN,-510.0,NaN,0.0,...,7109.0,5296.0,0.0,-1.0,-1.0,0.0,-3.0,1.0,45.0,-3.0
2024-09-26 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7109.0,5296.0,0.0,-1.0,-1.0,0.0,-3.0,94.0,45.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-26 23:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3417.0,1636.0,3.0,65.0,14.0,3.0,279.0,537.0,340.0,279.0
2024-09-26 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3417.0,1636.0,3.0,65.0,14.0,3.0,279.0,412.0,340.0,279.0
2024-09-26 23:45:00,NaN,0.0,NaN,0.0,NaN,0.0,NaN,-641.0,NaN,0.0,...,3483.0,1636.0,3.0,3.0,2.0,3.0,279.0,247.0,-146.0,279.0


In [19]:
# iterate through all days and files, collect dataframes, and append into a single dataset
dfs = []
for date_str in date_strings:
    dfs.append(
        extract_ES_data(
            HTML_STORAGE_PATH + "/" + date_str + EXTENSION,
            key_dict,
            pd.Timestamp(date_str),
        )
    )

# join data frames together
full_es_bids_keyed = pd.concat(dfs, ignore_index=True)

In [20]:
# Pivot the full_es_bids_keyed dataframe to have keys as columns and timestamps as index
full_es_bids = full_es_bids_keyed.pivot(index="datetime", columns="key", values="value")

# Ensure all keys from key_dict are present as columns, filling with NaN if missing
for key in key_dict.keys():
    if key not in full_es_bids.columns:
        full_es_bids[key] = np.nan

# Sort the columns to match the order in key_dict
full_es_bids = full_es_bids[list(key_dict.keys())]

# Sort the index (timestamps) in ascending order
full_es_bids.sort_index(inplace=True)

print(full_es_bids)

key                  tot_energy_ifm  tot_energy_ruc  tot_energy_rtpd  \
datetime                                                               
2022-07-31 00:00:00           170.0           170.0            185.0   
2022-07-31 00:05:00           170.0           170.0            185.0   
2022-07-31 00:10:00           170.0           170.0            185.0   
2022-07-31 00:15:00           170.0           170.0            114.0   
2022-07-31 00:20:00           170.0           170.0            114.0   
...                             ...             ...              ...   
2024-10-01 23:35:00           112.0           112.0           -447.0   
2024-10-01 23:40:00           112.0           112.0           -447.0   
2024-10-01 23:45:00           112.0           112.0           -684.0   
2024-10-01 23:50:00           112.0           112.0           -684.0   
2024-10-01 23:55:00           112.0           112.0           -684.0   

key                  tot_energy_rtd  tot_charge_ifm  tot_charge

In [21]:
# save full_es_bids to .parquet file
full_es_bids.to_parquet(BID_STORAGE_PATH + "/CAISO_ES_BIDS.parquet", engine="pyarrow")

# NOTE: Use the code snippet below if you want to read the parquet file:
# # example file reading
# full_es_bids = pd.read_parquet(BID_STORAGE_PATH + "/CAISO_ES_BIDS.parquet")